# Advanced Tutorial on MetaSynth

In this tutorial, we will be creating a `generative metadata format` (`gmf`) metadata file from a dataset using MetaSynth. We are going to walk through some of the advanced abilities of MetaSynth, such as handling dates, setting distributions and ensuring uniqueness in columns. This example workflow starts from a `.csv` file as input, but it easily adapted to other formats. 

You can run this notebook by checking out the MetaSynth repo and installing metasynth with `pip install metasynth`

In [1]:
# %pip install metasynth

In [2]:
# import required packages
import datetime as dt
import polars as pl
from metasynth import MetaDataset, demo_file

## Step 1: Transforming your data into a polars DataFrame

The first step in creating the metadata is reading and converting your dataset to a polars DataFrame. 

In [3]:
demonstration_fp = demo_file()
df = pl.read_csv(demonstration_fp, try_parse_dates=True, dtypes={
    "Sex": pl.Categorical,
    "Embarked": pl.Categorical})
df.head()

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],str
1,"""Braund, Mr. Ow…","""male""",22,0,7.25,null,"""S""",1937-10-28,15:53:04,2022-08-05 04:43:34,null
2,"""Cumings, Mrs. …","""female""",38,0,71.2833,"""C85""","""C""",null,12:26:00,2022-08-07 01:56:33,null
3,"""Heikkinen, Mis…","""female""",26,0,7.925,null,"""S""",1931-09-24,16:08:25,2022-08-04 20:27:37,null
4,"""Futrelle, Mrs.…","""female""",35,0,53.1,"""C123""","""S""",1936-11-30,null,2022-08-07 07:05:55,null
5,"""Allen, Mr. Wil…","""male""",35,0,8.05,null,"""S""",1918-11-07,10:59:08,2022-08-02 15:13:34,null


Now, let's check the data types of our DataFrame:

In [4]:
dict(zip(df.columns, df.dtypes))

{'PassengerId': Int64,
 'Name': Utf8,
 'Sex': Categorical,
 'Age': Int64,
 'Parch': Int64,
 'Fare': Float64,
 'Cabin': Utf8,
 'Embarked': Categorical,
 'Birthday': Date,
 'Board time': Time,
 'Married since': Datetime(time_unit='us', time_zone=None),
 'all_NA': Utf8}

We see that most variables are now nicely specified as strings, categories, dates and ints where necessary.

## Step 2: Creating a MetaDataset object from a DataFrame

Now a lot of work has already gone into creating a properly formatted dataframe. This work pays off at this stage: let's convert the DataFrame to a meta_dataset structure with the default options. Note: this takes a little bit of time!

In [5]:
meta_dataset = MetaDataset.from_dataframe(df)

/Users/qubix/Documents/work/metasynth/metasynth/metasynth/provider.py:202: UserWarning: 
Variable PassengerId seems unique, but not set to be unique.
Set the variable to be either unique or not unique to remove this warning.

  warnings.warn(f"\nVariable {series.name} seems unique, but not set to be unique.\n"


Then, we can show the metadata as a dictionary:

In [6]:
print(meta_dataset)

# Rows: 891
# Columns: 12

{'name': 'PassengerId', 'description': None, 'type': 'discrete', 'dtype': 'Int64', 'prop_missing': 0.0, 'distribution': "{'implements': 'core.discrete_uniform', 'provenance': 'builtin', 'class_name': 'DiscreteUniformDistribution', 'parameters': {'low': 1, 'high': 892}}"}

{'name': 'Name', 'description': None, 'type': 'string', 'dtype': 'Utf8', 'prop_missing': 0.0, 'distribution': '.[]{12,82}'}

{'name': 'Sex', 'description': None, 'type': 'categorical', 'dtype': 'Categorical', 'prop_missing': 0.0, 'distribution': "{'implements': 'core.multinoulli', 'provenance': 'builtin', 'class_name': 'MultinoulliDistribution', 'parameters': {'labels': array(['female', 'male'], dtype='<U6'), 'probs': array([0.35241302, 0.64758698])}}"}

{'name': 'Age', 'description': None, 'type': 'discrete', 'dtype': 'Int64', 'prop_missing': 0.19865319865319866, 'distribution': "{'implements': 'core.discrete_uniform', 'provenance': 'builtin', 'class_name': 'DiscreteUniformDistribution', 'p

## Step 3: Saving the metadata in a file

After creating the metadata, we can save it to a file. The default format is `json`, meaning the file is quite legible by humans and computers alike. Therefore, it can be checked by the data controller and, when the disclosure risk is deemed to be low, this file can be shared with others.

In [7]:
file_path = "demonstration_metadata.json"
meta_dataset.to_json(file_path)

## Step 4: Generating synthetic data from the metadata

Upon receiving this file, you can use the MetaSynth package to generate a synthetic version of the dataset:

In [8]:
new_meta_dataset = MetaDataset.from_json(file_path)
new_meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],f32
214,"""9"" w])z0#XuNK8…","""male""",19,0,19.020167,"""TLz~/=[4!Y/""","""S""",1932-11-07,15:06:07,2022-07-27 11:20:49,null
679,"""-A@C&s3kGrFlI""","""female""",null,2,22.278076,null,"""S""",1919-05-15,16:58:22,2022-07-25 23:06:38,null
21,"""\OqxLpRUb`pb&$…","""male""",50,2,1.349379,"""D27W""","""S""",1921-10-27,null,2022-08-09 01:03:33,null
107,"""tr:# NgUA,T xR…","""female""",69,0,77.426879,"""G=Bs[ V24]tqe#…","""S""",1905-02-18,11:17:27,2022-07-16 01:45:17,null
515,"""q}L+3 Q)1:C5D?…","""male""",13,0,67.02794,null,"""S""",1906-11-08,17:45:01,null,null


As you can see, the fake data looks a lot like the real data! However, it could still use some improvement. In the next sections, we will explore manual changes we can make to improve the quality of the synthetic data.

## Step 5: Improving the quality of the synthetic data

### Set unique columns

One column (PassengerId) has been detected as possibly unique by MetaSynth, as indicated by the following warning:

> "Variable PassengerId seems unique, but not set to be unique."

This column holds a variable with unique passenger identifiers, so in fact we do want synthetic data generated for this column to be unique as well. We can add this to the metadata by creating a list of options which we call a `specification`, or `spec`:

In [9]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}
}

# then, we add that dictionary as the `spec` argument
meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)

# then, let's check what the metadata about PassengerId contains!
meta_dataset["PassengerId"].to_dict()

{'name': 'PassengerId',
 'type': 'discrete',
 'dtype': 'Int64',
 'prop_missing': 0.0,
 'distribution': {'implements': 'core.unique_key',
  'provenance': 'builtin',
  'class_name': 'UniqueKeyDistribution',
  'parameters': {'low': 1, 'consecutive': 1}}}

So let's check what is generated from this new metadata:

In [10]:
meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],f32
1,"""&c{&oVHiru-x_H…","""female""",12,1,64.685147,"""AV0Hb, wo ""","""C""",1911-07-07,11:27:14,2022-07-29 07:59:06,null
2,""",=x/6ly}>d[ceK…","""female""",22,0,11.644796,null,"""S""",1928-09-20,17:17:14,2022-08-12 07:07:39,null
3,"""hfbf'Zc C$=:S[…","""male""",30,0,1.893007,null,"""S""",null,17:08:55,2022-08-03 22:43:55,null
4,"""qDjS6Y>}kl6Z' …","""male""",67,0,70.434857,null,"""S""",1940-05-22,13:13:42,2022-08-05 10:39:48,null
5,"""/@@tvl# Wi q~…","""male""",null,0,2.197158,null,"""C""",null,17:33:28,2022-08-01 01:17:44,null


Now we that the `PassengerId` column is correctly represented with increasing id numbers.

### Fake names (and others)

As one can see, the `Name` of the passengers is not quite so well synthesized. The reason is that the string type interpreter in MetaSynth is designed for `structured` strings (like room numbers such as `B1.09`, `B1.01` or `A1.08`) and not unstructured strings. However, MetaSynth supports the [faker](https://faker.readthedocs.io/en/master/index.html) package, which includes a lot of data types that it can fake. The columns using faker are not based on the real data at all so they do not disclose any info about the real data.

We fake names as follows:

In [11]:
# First, we create a specification dictionary for the variables
from metasynth.distribution import FakerDistribution

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": FakerDistribution("name")}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],f32
1,"""Carlos Kelly""","""male""",null,0,36.098234,null,"""S""",1920-01-16,15:11:33,2022-08-10 19:37:08,null
2,"""Tara Jackson""","""male""",50,0,51.496397,null,"""S""",1908-08-28,13:24:30,2022-08-11 21:29:41,null
3,"""John Hall""","""male""",51,0,59.436604,null,"""S""",1920-11-11,13:33:26,2022-08-05 11:51:47,null
4,"""Tina Coleman""","""male""",42,0,35.83177,null,"""C""",1940-02-22,12:11:58,2022-07-24 06:04:20,null
5,"""Erik Gentry""","""male""",31,0,5.418221,"""T~[<;""","""Q""",1930-12-15,16:40:35,2022-08-01 23:17:03,null


That already looks a lot better for the `Name` column!

### Set distributions manually

Without user input, the distribution chosen for each variable is inferred by choosing the best fitting from available distributions for the variable type. However, we can also manually specify which distribution to fit, or we can even just fully specify how the variable should be generated.

In [12]:
from metasynth.distribution import DiscreteUniformDistribution

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": FakerDistribution("name")},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit an exponential distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)} # fully specify a distribution for age (uniform between 20 and 40)
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],f32
1,"""Carlos Kelly""","""male""",38,0,0.877191,null,"""S""",1912-11-12,17:20:09,2022-08-12 22:25:55,null
2,"""Tara Jackson""","""male""",31,0,4.810884,null,"""S""",1912-08-27,17:55:01,2022-08-11 15:14:53,null
3,"""John Hall""","""male""",null,0,4.352428,null,"""C""",1932-07-13,11:00:45,2022-07-22 14:01:09,null
4,"""Tina Coleman""","""male""",25,0,0.198071,"""T..}X7""","""S""",1911-07-18,11:10:25,2022-07-27 17:43:37,null
5,"""Erik Gentry""","""male""",21,1,0.560526,"""A1\khuU5\FDj""","""S""",1934-04-16,10:51:19,2022-07-23 10:42:34,null


### Specifying the distribution of structured strings

For more or less structured strings, we can manually set the structure of the strings based on regular expressions. For example, we see that most Cabins are structured like [A-F] and then 2 or 3 digit numbers. We can include this as follows:

In [13]:
from metasynth.distribution import RegexDistribution

# To create a regex distribution, you need a list of tuples, where each tuple is an element.
# The first part of the tuple is a string representation of the regex, while the second is the proportion of the
# time the regex element is used.
cabin_distribution = RegexDistribution(r"[ABCDEF]\d{2,3}")  # Add the r so that it becomes a literal string.
# just for completeness: data generated from this distribution will always match the regex [ABCDEF]?(\d{2,3})?

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": FakerDistribution("name")},
    "Fare": {"distribution": "ExponentialDistribution"}, # estimate / fit an exponential distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)}, # fully specify a distribution for age (uniform between 20 and 40)
    "Cabin": {"distribution": cabin_distribution}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(10)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],f32
1,"""Carlos Kelly""","""female""",29,0,11.779226,null,"""S""",1905-06-15,15:52:19,2022-07-24 21:13:12,null
2,"""Tara Jackson""","""male""",38,0,16.717372,"""F932""","""S""",null,13:23:20,2022-08-12 03:33:40,null
3,"""John Hall""","""male""",34,0,21.10042,null,"""S""",1919-11-24,17:38:41,2022-08-07 15:14:21,null
4,"""Tina Coleman""","""male""",33,0,38.372983,null,"""S""",1938-09-24,12:35:36,2022-07-18 07:01:43,null
5,"""Erik Gentry""","""female""",30,1,30.103301,null,"""S""",1935-10-12,16:28:10,2022-08-09 14:26:30,null
6,"""William Davis""","""male""",21,0,37.616158,null,"""S""",1930-08-13,16:38:30,2022-08-12 12:41:44,null
7,"""Andrew Moreno""","""male""",37,0,24.799106,null,"""S""",1919-11-26,17:57:46,2022-07-17 23:24:14,null
8,"""Whitney Brown""","""male""",34,0,22.466841,null,"""S""",1919-07-10,11:40:18,2022-08-05 04:17:00,null
9,"""Jose Smith""","""male""",38,0,5.049875,null,"""S""",1933-03-06,16:53:17,2022-08-10 04:20:27,null


## Comparing the final synthetic dataset to the original

Let's first compare the averages of the numerical columns:

In [14]:
df.mean()

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
f64,str,cat,f64,f64,f64,str,cat,date,time,datetime[μs],str
446.0,null,null,29.693277,0.381594,32.204208,null,null,null,null,null,null


In [15]:
meta_dataset.synthesize(len(df)).mean()

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
f64,str,cat,f64,f64,f64,str,cat,date,time,datetime[μs],f32
446.0,null,null,29.707042,0.379349,29.770945,null,null,null,null,null,null


Then, we can also see how many missing values are in each column

In [16]:
df.null_count()

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,177,0,0,687,2,78,79,92,891


In [17]:
meta_dataset.synthesize(len(df)).null_count()

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,175,0,0,694,2,73,81,98,891
